### **Importing Necessary Libraries**

In [1]:
import cv2
import numpy as np
from pygame import mixer
from keras.models import load_model
import sys
sys.path.append('yoloface-master')
from face_detector import YoloDetector

pygame 2.5.2 (SDL 2.28.3, Python 3.12.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


### **Initializing the Model, Alarm Audio, Camera, and Text Font**

In [3]:
# Loading Our CNN Model
drowsiness_detector_model = load_model('model_30.keras', compile=False)

# Loading the Pretrained Face Detector Model
face_detector = YoloDetector(target_size=720, device = "cuda:0", min_face = 90)

# Initializing Pygame Mixer for Audio
mixer.init()
soundfile = 'C:/Users/Jamil/Downloads/Uni/GP/alarm.wav'
sound = mixer.Sound(soundfile)

# Initializing the Camera
cap = cv2.VideoCapture(0)

# Font for the text
font = cv2.FONT_HERSHEY_SIMPLEX

cuda:0
c:\Users\Jamil\Downloads\Uni\GP\Transfer Learning\Transfer\yoloface-master\weights/yolov5n_state_dict.pt


### **Initializing Necessary Variables for Detection**

In [4]:
# Score for the number of frames where the Driver is Drowsy
drowsy_score = 0

# Threshold for the number of frames where the Driver is Drowsy
drowsy_score_threshold = 20

# Flag for the Drowsiness status, Set to True if drowsy_score exceeds the threshold drowsy_score_threshold
drowsy = False

# Counter for the number of frames where the Face is Not Detected
face_not_detected_counter = 0

# Threshold for the number of frames the Face is Not Detected
face_not_detected_threshold = 20

# Flag for the Face Status
face_not_detected = False

# Flag for the Alarm Status
alarm_playing = False

# **Functions**

### **play_alarm()**

#### Plays the alarm indefinetely if it is not already playing

In [5]:
def play_alarm():
    global alarm_playing
    if not alarm_playing:
        sound.play(-1)
        alarm_playing = True

### **stop_alarm()**
#### Stops the alarm if it is playing

In [6]:
def stop_alarm():
    global alarm_playing
    if alarm_playing:
        sound.stop()
        alarm_playing = False                                                                              

### **check_face()**
#### Checks if the driver's face is visible

In [7]:
def check_face(faces):
    global face_not_detected, face_not_detected_counter, face_not_detected_threshold
    
    
    if faces:
        for face in faces[0]:
            # If a face is detected, reset the face_not_detected_counter to 0
            if face:
                face_not_detected_counter = 0
            
            # If no face is detected, increment the face_not_detected_counter by 1
            elif not face:
                face_not_detected_counter += 1

    # If the face_not_detected_counter exceeds the face_not_detected_threshold, set face_not_detected to True
    if face_not_detected_counter >= face_not_detected_threshold:
        face_not_detected = True
    else:
        face_not_detected = False

### **Drowsiness Detection Main Code**

In [9]:
# Start the Camera
cap = cv2.VideoCapture(0)

# This counter is used to calculate benchmarks every 600 frames
counter = 0

# Main Loop
while True:
    
    # Get frame
    ret, frame = cap.read()
    
    # Detect Faces using the Face Detector
    faces = face_detector.predict(frame)
    
    if faces:
        for face in faces[0]:
            
            # If face is not detected
            if not face:
                check_face(faces)
                
                if face_not_detected:
                    cv2.putText(frame, 'Face Not Detected', (50, 50), font, 1, (0, 0, 255), 2, cv2.LINE_AA)
                    
                    # Play alarm
                    if not alarm_playing:
                        play_alarm()
                else:
                    # Stop alarm
                    if alarm_playing:
                        stop_alarm()
                
            else:
                # Draw the rectangle around the face
                cv2.rectangle(frame, (int(face[0][0]), int(face[0][1])), (int(face[0][2]), int(face[0][3])), (0, 255, 0), 2)
                
                # Crop the face
                face = frame[int(face[0][1]):int(face[0][3]), int(face[0][0]):int(face[0][2])]
                
                # Resize the face to 64x64 (Expected by Our CNN Model)
                face = cv2.resize(face, (64, 64))
                
                # Normalize the face
                face = face/255
                
                # Expand the dimensions of the face
                face = np.expand_dims(face, axis=-1)
                face = np.expand_dims(face, axis=0)
                
                # Get the Prediction from Our CNN Model
                prediction = drowsiness_detector_model.predict(face)
                
                # If the prediction < 0.5 (Drowsy), increment the drowsy_score by 1
                if prediction < 0.5:
                    drowsy_score += 1
                    cv2.putText(frame, 'Drowsy', (50, 50), font, 1, (0, 0, 255), 2, cv2.LINE_AA)
                
                # If the prediction >= 0.5 (Awake), decrement the drowsy_score by 1
                elif prediction >= 0.5:
                    drowsy_score -= 1
                    cv2.putText(frame, 'Awake', (50, 50), font, 1, (0, 255, 0), 2, cv2.LINE_AA)
                
                # If the drowsy_score < 0, set it to 0
                if drowsy_score < 0:
                    drowsy_score = 0
                
                # Display the Drowsy Score
                cv2.putText(frame,f"Drowsy Score: {drowsy_score}", (50, 100), font, 1, (0, 0, 255), 2, cv2.LINE_AA)
                
                # If the drowsy_score exceeds the drowsy_score_threshold, set drowsy to True and play the alarm
                if drowsy_score >= drowsy_score_threshold:
                    drowsy = True
                    if not alarm_playing:
                        play_alarm()
                         
                # If the drowsy_score is less than the drowsy_score_threshold, set drowsy to False and stop the alarm
                else:
                    drowsy = False
                    if alarm_playing:
                        stop_alarm()
    
    # Show the Frame    
    cv2.imshow('Drowsiness Detection', frame)
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


# Release the Camera and Destroy the Windows
cap.release()
cv2.destroyAllWindows()

# Stop the alarm if it is playing
if alarm_playing:
    stop_alarm()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 422ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━